<a href="https://colab.research.google.com/github/fatou29-kine/Brain-Tumor-/blob/main/MLSS-DNN-DAY2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parallel and distributed Deep Learning

## Author: Marieme Ngom, Argonne National Laboratory
(combining and adapting materials/discussion evolved over time by Huihuo Zheng, Bethany Lusch, Asad Khan, Prasanna Balaprakash, Taylor Childers, Corey Adams, Kyle Felker, Varuni Sastry, Sam Foreman, Archit Vasan, Carlo Graziani, Tanwi Mallick, and Venkat Vishwanath)
## Outline
1. Day 1
    - Evolution of computig systems
    - Parallel computing
    - Introduction to Deep Learning
    - ***Parallel computing in AI***


2. ***Day 2***
    - ***Parallel computing in AI***
    - Brief Introduction to LLMs
    - Hands-on LLM training


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Parallel Computing in AI
### Recap Single GPU
![x0](https://github.com/mngom2/DNNMLSS/blob/main/images/mermaid-figure-1.png?raw=1)

Distributed training is the process of training models across multiple GPUs or other accelerators, with the goal of speeding up the training process and enabling the training of larger models on larger datasets.

There are two ways of parallelization in distributed training.
* ***Data parallelism***:
    * Each worker (GPU) has a complete set of model
    * different workers work on different subsets of data.
* *Model parallelism*
    * The model is split into different parts and stored on different workers
    * Different workers work on computation involved in different parts of the model
![PI](https://github.com/mngom2/DNNMLSS/blob/main/images/parallel_computing.png?raw=1)

## Scaling goal:
1. Minimize cost i.e. amount of time spent training
2. Maximize performance i.e model quality metrics, throughput/efficiency metrics (images/seconds, GPU/CPU utilization percentages, flops efficiency)

## Training on multiple GPUs: Data Parallelism
### Nomenclature:
- N = number of GPUs = WORLD_SIZE
- Each GPU is assigned a rank from 0 to WORLD_SIZE-1
- A worker = a GPU here
![mgpus](https://github.com/mngom2/DNNMLSS/blob/main/images/mermaid-figure-15.png?raw=1)
*Each GPU receives unique data at each step*
### Data Parallel: Forward Pass
![forward](https://github.com/mngom2/DNNMLSS/blob/main/images/mermaid-figure-14.png?raw=1)
*Average gradients across all GPUs*
### Data Parallel: Backward Pass
![backward](https://github.com/mngom2/DNNMLSS/blob/main/images/mermaid-figure-13.png?raw=1)
*Send global updates back to each GPU*
### Data Parallel: Full Setup
![full](https://github.com/mngom2/DNNMLSS/blob/main/images/mermaid-figure-12.png?raw=1)
*See: [PyTorch / Distributed Data Parallel](https://docs.pytorch.org/tutorials/intermediate/ddp_tutorial.html)*
### Data Parallel: Training
- Each GPU:
    - has identical copy of model
    - works on a unique subset of data
- Easy to get started with
    - [saforeman2/ezpz](https://github.com/saforem2/ezpz)
    - [PyTorch/DDP](https://docs.pytorch.org/docs/stable/notes/ddp.html)
    - [HF/Accelerate](https://huggingface.co/docs/transformers/accelerate)
    - [Microsoft/DeepSpeed](https://www.deepspeed.ai)
- Requires ***global*** communication
    - every rank must participate (collective communication)

## Communication
- Need mechanism(s) for communicating across GPUs:
    - [mpi4py](https://mpi4py.readthedocs.io/en/stable/tutorial.html)
    - [torch.distributed](https://docs.pytorch.org/docs/stable/distributed.html)
- Collective communication:
    - [Nvidia Collective Communications Library (NCCL)](https://developer.nvidia.com/nccl)
    - [Intel oneAPI Collective Communications](https://www.intel.com/content/www/us/en/developer/tools/oneapi/oneccl.html#gs.n9y302)
***Timeouts*** Collective operations have to be called for each rank to form a complete collective operation.
Failure to do so will result in other ranks waiting indefinitely
### AllReduce
![allreduce](https://github.com/mngom2/DNNMLSS/blob/main/images/mermaid-figure-11.png?raw=1)

### Broadcast
![broadcast](https://github.com/mngom2/DNNMLSS/blob/main/images/mermaid-figure-9.png?raw=1)

## Dealing with Data
- At each training step, we want to ensure that each worker receives unique data
- This can be done in one of two ways:
    1. Manually partition data (ahead of time)
        - Assign unique subsets to each worker
        - Each worker can only see their local portion of the data
        - Most common approach
    2. From each worker, randomly select a mini-batch
        - Each worker can see the full dataset
        - ⚠️ When randomly selecting, it is important that each worker uses different seeds to ensure they receive unique data

## Broadcast Initial State
- At the start of training (or when loading from a checkpoint), we want all of our workers to be initialized consistently
    - Broadcast the model and optimizer states from rank() == 0 worker
![bcast](https://github.com/mngom2/DNNMLSS/blob/main/images/mermaid-figure-6.png?raw=1)

## Why distributed training?
- N workers each processing unique batch (micro batch size) of data:
    - (micro_batch_size = 1)× $N_{GPUs}$ → ***global_batch_size = N***
- Improved gradient estimators
    - Smooth loss landscape
    - Less iterations needed for same number of epochs
        - common to scale learning rate lr *= sqrt(N)
        
![speedup](https://github.com/mngom2/DNNMLSS/blob/main/images/speedup.png?raw=1)

## Going Beyond Data Parallelism
- Useful when model fits on single GPU:
    - ultimately limited by GPU memory
    - model performance limited by size
- ⚠️ When model does not fit on a single GPU:
    - Offloading (can only get you so far…):
        - DeepSpeed + ZeRO
        - PyTorch + FSDP
- Otherwise, resort to [model parallelism strategies](https://samforeman.me/talks/ai-for-science-2024/slides#/additional-parallelism-strategies)


# Going beyond Data Parallelism:  DeepSpeed + ZeRO
- Depending on the ZeRO stage (1, 2, 3), we can offload
    1. ***Stage 1***: optimizer states (P_{os})
    2. ***Stage 2***: optimizer states+gradients (P_{os+g})
    2. ***Stage 3***: optimizer states+gradients+model params (P_{os+g+p})

![zero](https://github.com/mngom2/DNNMLSS/blob/main/images/zero.png?raw=1)

# Model parallel training: example
Want to compute $y = \sum_i x_iW_i = x_0W_0 + x_1W_1 + x_2W_2$ where each GPU only has only its portion of the full weights as shown below
1. Compute $y_0=x0W_0$ -> **GPU1**
2. Compute $y_1=y_0 +x_1W_1$ -> **GPU2**
3. Compute $y_2=y_1 + x_2W_2$

![modelpar](https://github.com/mngom2/DNNMLSS/blob/main/images/mermaid-figure-2.png?raw=1)

# Deciding on a parallelism strategy
![onedec](https://github.com/mngom2/DNNMLSS/blob/main/images/onegpudec.png?raw=1)
![multgpu](https://github.com/mngom2/DNNMLSS/blob/main/images/onenodemulgpu.png?raw=1)

![AIcompute](https://github.com/mngom2/DNNMLSS/blob/main/images/ai-and-compute-all-2.png.webp?raw=1)

Sophia: 192 GPUs (8/node), 3.9 Petaflops ($10^15$)/s
![sophia](https://github.com/mngom2/DNNMLSS/blob/main/images/sophia.jpeg?raw=1)
Polaris: 2240 GPUs (4/node), 78 Teraflops ($10^12$)/s
![polaris](https://github.com/mngom2/DNNMLSS/blob/main/images/polaris.jpeg?raw=1)
Aurora: 63,744 GPUs (6/node), exascale computer ($10^18$ calculations per second)
![aurora](https://github.com/mngom2/DNNMLSS/blob/main/images/aurora.jpeg?raw=1)

# Brief introduction to LLMs

## Training LLMs

## Life-cycle of a LLM
1. Data collection + preprocessing
2. ***Pre-training***
    - Architecture decisions, model size, etc.
3. Supervised Fine-Tuning
    - Instruction Tuning
    - Alignment
4. Deploy (+ monitor, re-evaluate, etc.)

![gptcycle](https://github.com/mngom2/DNNMLSS/blob/main/images/gpt3-training-step-back-prop.gif?raw=1)
*Source:Figure from [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)*

## Life-cycle of a LLM
1. Data collection + preprocessing
2. Pre-training
    - Architecture decisions, model size, etc.
3. ***Supervised Fine-Tuning***
    - Instruction Tuning
    - Alignment
4. Deploy (+ monitor, re-evaluate, etc.)

![gptcycle2](https://github.com/mngom2/DNNMLSS/blob/main/images/gpt3-fine-tuning.gif?raw=1)
*Source:Figure from [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)*

## Forward pass
![fwdpass](https://github.com/mngom2/DNNMLSS/blob/main/images/hf_assisted_generation.mov?raw=1)
*Source: [Generation with LLMs](https://huggingface.co/docs/transformers/main/en/llm_tutorial)*

## Generating text
![fwdpass](https://github.com/mngom2/DNNMLSS/blob/main/images/hf_assisted_generation2.mov?raw=1)
*Source: [Generation with LLMs](https://huggingface.co/docs/transformers/main/en/llm_tutorial)*

# Hands-on LLM Training


***Good practice*** (not needed here): Create and activate a conda (or virtual) environment
```conda create -n env_mlss_dnn python=3.9```
then on jupyter do new ->terminal

```
 conda activate env_mlss_dnn
 pip install ipykernel
 python -m ipykernel install --user --name env_mlss_dnn
```

then go back to your .ipynb file, change kernel to env_mlss_dnn.



In [2]:
!git clone https://github.com/karpathy/nanoGPT.git

Cloning into 'nanoGPT'...
remote: Enumerating objects: 686, done.
remote: Total 686 (delta 0), reused 0 (delta 0), pack-reused 686 (from 1)
Receiving objects: 100% (686/686), 954.04 KiB | 12.08 MiB/s, done.
Resolving deltas: 100% (387/387), done.


In [3]:
%pwd

%cd nanoGPT

%pwd

/content/nanoGPT


'/content/nanoGPT'

In [4]:
!pip install torch numpy transformers datasets tiktoken wandb tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

| Dataset               | Tokens (≈)         | Disk size / notes                                 |
| --------------------- | ------------------ | ------------------------------------------------- |
| **openwebtext**       | 9 B tokens total   | 9 B train (≈17GB) / 4M val (≈8.5MB)                |
| **shakespeare (tiny)** | ≈ 330K tokens total | 301,966 train / 36,059 val                   |
| **shakespeare\_char** | 1,115,394 chars    | 1,003,854 train / 111,540 val (character‐level)   |


| model | params | train loss | val loss |
| ------| ------ | ---------- | -------- |
| gpt2 | 124M         | 3.11  | 3.12     |
| gpt2-medium | 350M  | 2.85  | 2.84     |
| gpt2-large | 774M   | 2.66  | 2.67     |
| gpt2-xl | 1558M     | 2.56  | 2.54     |


In [5]:
!python3 data/shakespeare_char/prepare.py

length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


In [7]:
!python3 train.py config/train_shakespeare_char.py --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=16 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

In [8]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ayopwtkc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ayopwtkc
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803707 sha256=eb9b6226da8a3d1efb23d40dc8d0e1fcbcef421d1f29b0b4efdd85d0ed69f7ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-qsmrqsz0/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built openai-whisper


In [9]:
import tiktoken

In [10]:
!python3 sample.py --out_dir=out-shakespeare-char

Overriding: out_dir = out-shakespeare-char
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Traceback (most recent call last):
  File "/content/nanoGPT/sample.py", line 38, in <module>
    checkpoint = torch.load(ckpt_path, map_location=device)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 1425, in load
    with _open_file_like(f, "rb") as opened_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 751, in _open_file_like
    return _open_file(name_or_buffer, mode)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 732, in __init__
    super().__init__(open(name, mode))
                     ^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] N

In [11]:
!python3 train.py config/train_shakespeare_char.py #training longer

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

In [12]:
!python3 sample.py --out_dir=out-shakespeare-char

Overriding: out_dir = out-shakespeare-char
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Traceback (most recent call last):
  File "/content/nanoGPT/sample.py", line 38, in <module>
    checkpoint = torch.load(ckpt_path, map_location=device)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 1425, in load
    with _open_file_like(f, "rb") as opened_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 751, in _open_file_like
    return _open_file(name_or_buffer, mode)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 732, in __init__
    super().__init__(open(name, mode))
                     ^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] N

In [13]:
!export NCCL_DEBUG=INFO
!export NCCL_DEBUG_SUBSYS=ALL
!export NCCL_DEBUG_FILE=nccl_trace.log

# Running on NVIDIA T4 Tensor Cores, 4GPUS/node

In [14]:
import socket
ip=socket.gethostbyname(socket.gethostname())
print(ip)

172.28.0.12


In [15]:
!export CUDA_VISIBLE_DEVICES=0 #,1,2,3
!export MASTER_ADDR=ip
!export MASTER_PORT=29500

!torchrun \
  --nnodes=1 \
  --node_rank=0 \
  --nproc_per_node=12 \
  --master_addr=$ip \
  --master_port=29500 \
  train.py \
    config/train_shakespeare_char.py \
    --batch_size=64 \
    --gradient_accumulation_steps=40

W0625 17:31:09.925000 19317 torch/distributed/run.py:792] 
W0625 17:31:09.925000 19317 torch/distributed/run.py:792] *****************************************
W0625 17:31:09.925000 19317 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0625 17:31:09.925000 19317 torch/distributed/run.py:792] *****************************************
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command li

![llms](https://github.com/mngom2/DNNMLSS/blob/main/images/llms.gif?raw=1)
*Source: [Hannibal046/Awesome-LLM](https://github.com/Hannibal046/Awesome-LLM)*

![emergent](https://github.com/mngom2/DNNMLSS/blob/main/images/emergent-abilities.gif?raw=1)


![evolllms](https://github.com/mngom2/DNNMLSS/blob/main/images/evolution.gif?raw=1)